In [1]:

import pandas as pd

# Creamos y procesamos las tablas necesarias para FastAPI

## Abrimos tablas procesadas anteriormente

- user_review_NLP.parquet

In [2]:
# Direccion del archivo comprimido y reconvertido
user_reviews_content = '_data/02_user_reviews_NLP.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
user_reviews_dataset = pd.read_parquet(user_reviews_content)

- users_items_final.parquet

In [3]:
# Direccion del archivo comprimido y reconvertido
users_items_content = '_data/02_users_items_final.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
users_items_dataset = pd.read_parquet(users_items_content)

- steam_games_final.parquet

In [4]:
# Direccion del archivo comprimido y reconvertido
steam_games_content = '_data/02_steam_games_FastAPI.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
steam_games_dataset = pd.read_parquet(steam_games_content)

## Trabajamos armando una tabla combinando los datos de las tablas

### A continuacion utilizamos: 

- 'app_name', 'id' y 'release_year' de steam_games_dataset 
- 'item_id, 'recommend' y 'sentiment_analysis' de user_reviews_dataset

Esta tabla servirá para las funciones de FastAPI:
- UsersRecommend
- UsersNotRecommend
- sentiment_analysis

Primero renombramos 'id' de steam_games_dataset para que se corresponda con 'item_id' de user_reviews_dataset. 

In [5]:
steam_games_dataset.rename(columns={'id': 'item_id'}, inplace=True)

In [6]:
# Fusionar los DataFrames por las columnas mencionadas
merged_steam_rev_data = pd.merge(steam_games_dataset[['app_name', 'item_id', 'release_year']],
                                  user_reviews_dataset[['item_id', 'recommend', 'sentiment_analysis']],
                                  on='item_id')

# Ordenar por la columna 'app_name'
merged_steam_rev_data.sort_values('release_year', inplace=True)


In [7]:
merged_steam_rev_data.head(2)

,app_name,item_id,release_year,recommend,sentiment_analysis
14190,Sword of the Samurai,327950,1989,True,2
43573,LOOM™,32340,1990,True,2


In [8]:
merged_steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49057 entries, 14190 to 20550
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   app_name            49057 non-null  object
 1   item_id             49057 non-null  int64 
 2   release_year        49057 non-null  object
 3   recommend           49057 non-null  bool  
 4   sentiment_analysis  49057 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 1.9+ MB


In [9]:
print(f"El DataFrame tiene {merged_steam_rev_data.shape[0]} filas y {merged_steam_rev_data.shape[1]} columnas.")

El DataFrame tiene 49057 filas y 5 columnas.


In [10]:
merged_steam_rev_data.drop_duplicates(inplace=True)

In [11]:
merged_steam_rev_data['release_year'] = merged_steam_rev_data['release_year'].astype(int)

### Controlamos el nuevo DataFrame

In [12]:
merged_steam_rev_data.head(2)

,app_name,item_id,release_year,recommend,sentiment_analysis
14190,Sword of the Samurai,327950,1989,True,2
43573,LOOM™,32340,1990,True,2


In [13]:
merged_steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6288 entries, 14190 to 19447
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   app_name            6288 non-null   object
 1   item_id             6288 non-null   int64 
 2   release_year        6288 non-null   int64 
 3   recommend           6288 non-null   bool  
 4   sentiment_analysis  6288 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 251.8+ KB


In [14]:
print(f"El DataFrame tiene {merged_steam_rev_data.shape[0]} filas y {merged_steam_rev_data.shape[1]} columnas.")

El DataFrame tiene 6288 filas y 5 columnas.


In [15]:
# Guardar el DataFrame fusionado en un archivo .parquet
merged_steam_rev_data.to_parquet('FastAPI/merged_steam_rev_data.parquet', index=False)

### Ahora utilizamos: 

- 'item_id', 'user_id' y 'playtime_forever' de users_items_dataset 
- 'genres' y 'release_year' de steam_games_dataset

Esta tabla servirá para las funciones de FastAPI:
- PlayTimeGenre
- UserForGenre

In [16]:
# Fusionar los DataFrames por las columnas mencionadas
merged_steam_user_data = pd.merge(users_items_dataset[['item_id','user_id', 'playtime_forever']], steam_games_dataset[['genres', 'release_year']], left_on='item_id', right_index=True)

# Ordenar por la columna 'genres'
merged_steam_user_data.sort_values('playtime_forever', inplace=True)

### Controlamos el nuevo DataFrame

In [17]:
merged_steam_user_data.head()

,item_id,user_id,playtime_forever,genres,release_year
2892317,21090,nolongerliving,0,Simulation Strategy,2012
2851830,12800,daymm,0,Simulation,2017
2782437,12800,76561198019707497,0,Simulation,2017
2716971,12800,76561198066131731,0,Simulation,2017
3781460,13210,76561197995292266,0,Adventure Casual,2014


In [18]:
print(f"El DataFrame tiene {merged_steam_user_data.shape[0]} filas y {merged_steam_user_data.shape[1]} columnas.")

El DataFrame tiene 1074137 filas y 5 columnas.


In [19]:
nan_present = merged_steam_user_data.isna().any().any()

if nan_present:
    print("Se encontraron valores NaN en el DataFrame.")
else:
    print("No se encontraron valores NaN en el DataFrame.")

No se encontraron valores NaN en el DataFrame.


In [20]:
merged_steam_user_data.drop_duplicates(inplace=True)

In [21]:
print(f"El DataFrame tiene {merged_steam_user_data.shape[0]} filas y {merged_steam_user_data.shape[1]} columnas.")

El DataFrame tiene 1062375 filas y 5 columnas.


In [22]:
merged_steam_user_data['release_year'] = merged_steam_user_data['release_year'].astype(int)

In [23]:
merged_steam_user_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1062375 entries, 2892317 to 587715
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   item_id           1062375 non-null  int64 
 1   user_id           1062375 non-null  object
 2   playtime_forever  1062375 non-null  int64 
 3   genres            1062375 non-null  object
 4   release_year      1062375 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 48.6+ MB


## Randomizacion y reducción

- Viendo que la tabla creada es extensa, creamos una mas pequeña tomando filas aleatoriamente. Para que esta sea lo suficientemente liviana y manejable por FastAPI y la matriz de recomendaciones, la reducimos aproximadamente al 3%, que se asemeja al tamaño de la tabla merged_steam_rev_data.

In [24]:
frac=int(merged_steam_user_data.shape[0]*0.003) # Indice para obtener la fraccion de reduccion
merged_steam_user_data = merged_steam_user_data.sample(frac, random_state=62)  # 'random_state' para reproducibilidad

- Se guarda la nueva tabla

In [25]:
# Guardar el DataFrame fusionado en un archivo .parquet
merged_steam_user_data.to_parquet('FastAPI/merged_steam_user_data.parquet', index=False)